### Bonferroni's Principle

Suppose you have a certain amount of data, and you look for events of a certain type within that data. You can expect events of this type to occur, even if the data is completely random, and the number of occurrences of these events will grow as the size of the data grows. These occurrences are “bogus,” in the sense that they have no cause other than that random data will always have some number of unusual features that look significant but aren’t.


A theorem of statistics, known as the **Bonferroni correction** gives a statistically sound way to avoid most of these bogus positive responses to a search through the data. Without going into the statistical details, we offer an informal version, Bonferroni’s principle, that helps us avoid treating random occurrences as if they were real.

### Large Scale File System organization

This new file system is called distributed file system (DFS) and is used for:
    
    - Enormous files, possibly in terrabyte in size. For, small files have no point to use DFS 
    - Files are rarely updated. They are used to read data or appending additional data.
    

Files are divided into chunks (typically 64 MB). Chunks are replicated (~3 times) on different compute nodes (should be different racks). To find the chunk of file, there is another small file master node or name node for that file. Master node is itself replicated.


**Chunk Servers**
1. When a map reduce job is started, chunks are distributed among different worker nodes/ chunk servers with some replication factor
2. Each chunk server can have multiple chunks of data, with each chunk having its own mapper.

### Map Reduce

* Manages large scale computations in a way that is tolerant of hardware faults.
* 2 Functions, Map and Reduce, while system manages the parallel execution.

**Steps for map-reduce computation**
1. Some number of Map tasks each are given one or more chunks from a distributed file system. These map tasks turns the chunk into a sequence of key-value pairs.
2. Key-Value pairs from each map task are collected by a master controller and sorted by key. The key are divided among all the reduce tasks. All key-value pairs with same key wind up at same Reduce task.
3. The Reduce tasks work on one key at a time and combine all the values associated with the key in some way.


* **Map Task**
* **Grouping and Aggregation**
* **Reduce Task**
        
**Coping with Failures**
1. If node at with Master is executing fails, then the entire map-reduce job must be restarted. This is the worst case, other failures will be managed by the Master, and map-reduce job will complete eventually.
2. Suppose if Map worker fails. The fail will be detected by Master because it periodically pings the Workers. All map task assigned to this worker will have to redone even if it is completed. Becuase output is destined to Reduce tasks, and now unavailable to reduce tasks. The master must also inform reduce tasks that the location of input from map task has changed.
3. Dealing with failure at the node of Reduce worker is simpler. The Master simply sets the status of its currently executing Reduce tasks to idle. These will be rescheduled on another reduce worker later.

### Data Flow
1. Input and output data is stored on DFS. Scheduler tries to map tasks "close" to physical storage (chunk servers) of input data.
2. Intermediate results are stored on local FS of Map and reduce workers
3. Output is often input to another MapReduce task.

### Coordination : Master
1. Check status of task (idle, in-progress, completed)
2. idle - no worker assigned, gets scheduled as worker become available
3. When map tasks completes, it sends the master the location/sizes of intermediate files, one for each reducer.
4. Master pings workers periodically to detect failures.

**Task Granularity :** Measures amount of work which is performed by task.

**Fine Granularity tasks :** Finer grains -> more parallelism -> speedup. Minimizes time for fault recovery, pipeline shuffling, better load balancing.

**Refinement (Backup tasks) :** Slow worker -> increases job completion time. **Solution** is to keep backup copies of tasks. Whichever finishes first "wins". **Effect** Dramatically shortens job completion time.

**Refinement (Combiners) :** It is common for reduce function to be associate and commulative. So, it is possible to push some of what reduce does to Map tasks. Example instead of emitting (w,1) we could apply reduce function within the Map tasks, before it is subjected to grouping.

**Refinement (partition) :** Controlling how keys get partitioned, override the default partition function. Reducer need to ensure that record with same intermediate key end up at the same worker.

### Algorithms using Map-Reduce

Map-Reduce is not a solution to every problem, not even every problem that profitable can use many compute nodes operating in parellel.

1. **Matrix-Vector Multiplication**
   We have Matrix M (n x n) where $m_{ij}$ denote element at row i and column j. We also have a vector v of length n, jth element is $v_j$. Element $x_i$ is given by.
$$ x_i = \sum_{j=1}^n m_{ij}v_j $$
    * The matrix will be stored in DFS in the form of triples (i,j, $m_{ij}$).
    * *Map Function*: Each Map task will take entire vector v and a chunk of matrix M, to produce a key value pair. (i, $m_{ij}v_j$).
    * *Reduce Function*: A reduce task has simply to sum all values associated with a given key i. The result will be pair (i,$x_i$).

2. **Computing Selections**
    Do not need full power of map-reduce. Can be done in map portion alone.
    * *Map Function*: For each tuple t in R test if it satisfies C. If so, produce the key-value pair (t,t).
    * *Reduce Function*: Simply passes each key-value pair to the output.

3. **Computing Projections**
    Similar to selection, because may cause duplicates, reduce function must eliminate duplicates.
    * *Map Function*: For each tuple t in R, construct t'. Output the key-value pair (t', t').
    * *Reduce Function*: For each key t' just output one t'.


4. **Union**
    * *Map Function*: Turn each input tuple t into a key-value pair (t,t).
    * *Reduce Function*: Associated with each key t there will be either one/two values. Produce output (t,t) in either case.


5. **Intersection**:
    * *Map Function*: Turn each tuple t into a key-value pair (t,t).
    * *Reduce Function*: If key t has value list [t,t], then produce (t,t). Other wise produce (t, NULL)


6. **Difference**:
    * *Map Function*: For a tuple t in R, produce key-value pair (t,R) or (t,S).
    * *Reduce Function*: If assoicated value list is [R], then produce (t,t). else produce (t,NULL).

7. **Natural Join**
    * *Map Function*: For each tuple (a,b) of R, produce (b, (R,a)) as key-value pair. Similarly for S.
    * *Reduce Function*: Each key value b will be associated with a list of pairs that are either of the form (R,a) or (S,c). The outptut will be (b, (a,b,c)).
    
8. **Two Phase Map Reduce Matrix Multiply**
    * *1st Map Function*

     $$ A[ij] = emit( j, (A, i, A[ij]) ) $$ $$ B[jk] = emit( j, (B, k, B[jk] ) ) $$
    
    * *1st Reduce Function*
     
     For each (i,k) from A and B i.e (A, i, A[ij]) and (B, k, B[jk])
    
     $$ emit( (i,k), A[ij]*B[jk] ) $$
     
    * *2nd Map Function*
     
     pass through, does nothing
     $$ emit( k,v ) $$
     
    * *2nd Reduce Function*
     
     $$ emit( (i,k), sum(values) ) $$